# 2022年5月17日 Julia学习笔记
## 使用参数化类型
Julia语言最强大的功能之一就是能够对类型进行参数化。参数化类型允许软件设计人员归纳类型，并让Julia运行时根据指定的参数自动编译为具体类型。

### 使用参数化复合类型


首先回顾上一次内容中建立的股票类型

In [2]:
abstract type Asset end

abstract type Property <: Asset end
abstract type Investment <: Asset end
abstract type Cash <: Asset end

abstract type House <: Property end
abstract type Apartment <: Property end

abstract type FixedIncome <: Investment end
abstract type Equity <: Investment end

struct Stock <: Equity 
    symbol::String
    name::String
end

如果希望追踪自己拥有的股票数量，为此，定义一个名为StockHolding的新类型，如下：

In [2]:
struct StockHolding
    stock::Stock
    quantity::Int
end

如果需要数量中支持小数，可以将quantity类型改为Float64：

In [2]:
struct StockHolding
    stock::Stock 
    quantity::Float64
end

如果程序中需要同时支持Int和Float64类型，那么必须维护两个略有不同的类型。灵活性太差，而且可能会比较悲催。可以使用参数重新设计StockHolding类型：

In [5]:
struct StockHolding{T}
    stock::Stock
    quantity::T
end

T称为类型参数。作为占位符，可以在任何字段中用作类型。于是StockHolding{Int}类型指包含Int类型的quantity字段的类型，同样，StockHolding{Float64}指包含Float64类型的quantity字段的类型。
实际上，这个例子中，T类型只能是数字，因此可以进一步限定T为Real的任何子类型，即：

In [3]:
struct StockHolding{T <: Real}
    stock::Stock 
    quantity::T
end

In [8]:
stock = Stock("APPL", "Apple, Inc.")
holding = StockHolding(stock, 100)

StockHolding{Int64}(Stock("APPL", "Apple, Inc."), 100)

In [7]:
holding = StockHolding(stock, 100.0)

StockHolding{Float64}(Stock("APPL", "Apple, Inc."), 100.0)

In [9]:
holding = StockHolding(stock, 100//3)

StockHolding{Rational{Int64}}(Stock("APPL", "Apple, Inc."), 100//3)

参数类型的另一种用途是强制字段类型的一致性。假设我们要设计另一种所持股票对象来追踪所持股票的价格和市场价值。如下：

In [12]:
struct StockHolding2{T <: Real, P <: AbstractFloat}
    stock::Stock
    quantity::T
    price::P
    marketvalue::P
end

由于quantity类型与price和marketvalue的类型不同，因此我们添加了一个新的类型参数P。调用如下：

In [14]:
holding = StockHolding2(stock, 100, 180.00, 18000.00)

StockHolding2{Int64, Float64}(Stock("APPL", "Apple, Inc."), 100, 180.0, 18000.0)

In [15]:
holding = StockHolding2(stock, 100, 180.00, 18000)

LoadError: MethodError: no method matching StockHolding2(::Stock, ::Int64, ::Float64, ::Int64)
[0mClosest candidates are:
[0m  StockHolding2(::Stock, ::T, ::P, [91m::P[39m) where {T<:Real, P<:AbstractFloat} at In[12]:2

可以发现如果输入的marketvalue值不符合要求的类型，因此返回了一个错误。

参数类型也可以是抽象的。

### 2.6.2 使用参数化抽象类型
复合类型可以以参数化的方式进行增强，抽象类型同样也可以。比如：

In [17]:
abstract type Holding{P} end

然后Holding{P}的每个子类型还必须带有一个P类型参数。例如我们可以创建两个新类型StockHolding3{T, P}和CashHolding{P}:


In [19]:
struct StockHolding3{T, P} <: Holding{P}
    stock::Stock
    quantity::T
    price::P
    marketvalue::P
end

struct CashHolding{P} <: Holding{P}
    currency::String
    amount::P
    marketvalue::P
end

可以见擦这些类型之间的关系，如下所示：

In [20]:
StockHolding3{Int64, Float64} <: Holding{Float64}

true

In [21]:
certificate_in_the_safe = StockHolding3(stock, 100, 180.00, 18000.00)

StockHolding3{Int64, Float64}(Stock("APPL", "Apple, Inc."), 100, 180.0, 18000.0)

In [22]:
certificate_in_the_safe isa Holding{Float64}

true

在对类型进行参数化时，每个变体都被视为与其他变体无关的单独类型，只是它们具有共同的超类型。例如：Holding{Int}与Holding{Float64}是不同的类型，但是它们都是Holding的子类型。

In [24]:
Holding{Float64} <: Holding

true

In [25]:
Holding{Int} <: Holding

true

Julia附带一个非常丰富的类型系统，开发人员可以使用它来推理每种类型与其他类型的关系。
- 抽象类型使我们能够定义关系层次结构中的行为。
- 具体类型则用于定义数据的存储方式。
- 参数化类型用于将现有类型扩展为字段类型的变体。